In [13]:
%run change_data.py

Processing chunk starting at index 0
Processing chunk starting at index 1000
Processing chunk starting at index 2000
Processing chunk starting at index 3000
Processing chunk starting at index 4000
Processing chunk starting at index 5000
Processing chunk starting at index 6000
Processing chunk starting at index 7000
Processing chunk starting at index 8000
Processing chunk starting at index 9000
Processing chunk starting at index 10000
Processing chunk starting at index 11000
Processing chunk starting at index 12000
Processing chunk starting at index 13000
Processing chunk starting at index 14000
Processing chunk starting at index 15000
Processing chunk starting at index 16000
Processing chunk starting at index 17000
Processing chunk starting at index 18000
Processing chunk starting at index 19000
Processing chunk starting at index 20000
Processing chunk starting at index 21000
Processing chunk starting at index 22000
Processing chunk starting at index 23000
Processing chunk starting at 

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import stats
from scipy.stats import ttest_ind
import seaborn as sns

# --- CONFIG ---
RESULTS_DIR = "results"
FC_CUTOFF = 1.0         # |log2FC| threshold
PADJ_CUTOFF = 0.05
TOP_N = 50

Path(RESULTS_DIR).mkdir(exist_ok=True, parents=True)

# --- Load the data (same as in Part 2) ---
df = pd.read_csv("gene.tsv", sep='\t', index_col=0)
metadata = pd.read_csv("SRP181622/metadata_SRP181622.tsv", sep='\t', index_col=0)

print(f"Expression matrix shape: {df.shape}")
print(f"Metadata shape: {metadata.shape}")

# --- Create sample groups based on your Part 2 logic ---
sample_type = []
sample_names = []

for sample in metadata.index:
    if sample in df.columns:
        sample_names.append(sample)
        if 'Control' in metadata['refinebio_title'][sample]:
            sample_type.append('Control')
        else:
            sample_type.append('Stress')

# Create a metadata dataframe aligned with expression data
meta_aligned = pd.DataFrame({
    'sample': sample_names,
    'group': sample_type
}).set_index('sample')

# Align expression data to only include samples we have metadata for
expr = df[sample_names].copy()

print(f"Aligned expression matrix shape: {expr.shape}")
print(f"Groups: {meta_aligned['group'].value_counts()}")

REF_LEVEL = "Control"
CASE_LEVEL = "Stress"

# --- Prepare data for differential expression ---
# Get samples for each group
control_samples = meta_aligned[meta_aligned['group'] == REF_LEVEL].index.tolist()
stress_samples = meta_aligned[meta_aligned['group'] == CASE_LEVEL].index.tolist()

print(f"Control samples: {len(control_samples)}")
print(f"Stress samples: {len(stress_samples)}")

# --- Perform t-tests for each gene ---
results = []

for gene in expr.index:
    control_values = expr.loc[gene, control_samples].values
    stress_values = expr.loc[gene, stress_samples].values
    
    # Remove any NaN values
    control_values = control_values[~np.isnan(control_values)]
    stress_values = stress_values[~np.isnan(stress_values)]
    
    if len(control_values) > 1 and len(stress_values) > 1:
        # Perform t-test
        stat, pvalue = ttest_ind(stress_values, control_values, equal_var=False)
        
        # Calculate fold change (log2)
        mean_control = np.mean(control_values)
        mean_stress = np.mean(stress_values)
        
        # Add small constant to avoid log(0)
        log2fc = np.log2((mean_stress + 0.001) / (mean_control + 0.001))
        
        results.append({
            'gene': gene,
            'log2fc': log2fc,
            'pvalue': pvalue,
            'mean_control': mean_control,
            'mean_stress': mean_stress
        })

# Convert to DataFrame
res = pd.DataFrame(results)
res = res.set_index('gene')

# --- Apply Benjamini-Hochberg correction ---
from scipy.stats import false_discovery_control

# Handle any NaN p-values
res = res.dropna(subset=['pvalue'])

# Apply FDR correction
res['padj'] = false_discovery_control(res['pvalue'], method='bh')

# Sort by adjusted p-value
res = res.sort_values('padj')

print(f"Total genes analyzed: {len(res)}")
print(f"Significant genes (padj < {PADJ_CUTOFF}): {sum(res['padj'] < PADJ_CUTOFF)}")
print(f"Significant genes with |log2FC| >= {FC_CUTOFF}: {sum((res['padj'] < PADJ_CUTOFF) & (res['log2fc'].abs() >= FC_CUTOFF))}")

# --- Save results ---
full_path = os.path.join(RESULTS_DIR, "DE_results.csv")
res.to_csv(full_path)
print(f"Saved: {full_path}")

# --- Top N table ---
topN = res.head(TOP_N)
top_path = os.path.join(RESULTS_DIR, f"top_{TOP_N}_DEGs.csv")
topN.to_csv(top_path)
print(f"Saved: {top_path}")

# Display top 10 results
print("\nTop 10 differentially expressed genes:")
print(topN.head(10)[['log2fc', 'pvalue', 'padj', 'mean_control', 'mean_stress']].round(4))

# --- Volcano plot ---
x = res['log2fc']
y = -np.log10(res['padj'].clip(lower=1e-300))  # Avoid log(0)

# Define significant genes
sig = (res['padj'] < PADJ_CUTOFF) & (res['log2fc'].abs() >= FC_CUTOFF)
up_reg = (res['padj'] < PADJ_CUTOFF) & (res['log2fc'] >= FC_CUTOFF)
down_reg = (res['padj'] < PADJ_CUTOFF) & (res['log2fc'] <= -FC_CUTOFF)

plt.figure(figsize=(10, 8))
plt.scatter(x[~sig], y[~sig], s=20, alpha=0.5, color='gray', label='Not significant')
plt.scatter(x[up_reg], y[up_reg], s=20, color='red', label=f'Up-regulated ({sum(up_reg)})')
plt.scatter(x[down_reg], y[down_reg], s=20, color='blue', label=f'Down-regulated ({sum(down_reg)})')

plt.axvline(FC_CUTOFF, color='black', linestyle='--', alpha=0.7)
plt.axvline(-FC_CUTOFF, color='black', linestyle='--', alpha=0.7)
plt.axhline(-np.log10(PADJ_CUTOFF), color='black', linestyle='--', alpha=0.7)

plt.xlabel('log2 Fold Change (Stress vs Control)')
plt.ylabel('-log10(Adjusted P-value)')
plt.title('Volcano Plot: Differential Gene Expression')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()

volcano_path = os.path.join(RESULTS_DIR, "volcano_plot.png")
plt.savefig(volcano_path, dpi=300, bbox_inches='tight')
plt.show()
print(f"Saved: {volcano_path}")

# --- Heatmap of top significant genes ---
sig_genes = res.index[(res['padj'] < PADJ_CUTOFF) & (res['log2fc'].abs() >= FC_CUTOFF)].tolist()

if len(sig_genes) == 0:
    print("No genes passed both thresholds; using top 20 by p-value for heatmap.")
    sig_genes = res.head(20).index.tolist()

if len(sig_genes) > 50:
    sig_genes = sig_genes[:50]  # Limit to top 50 for readability

print(f"Creating heatmap with {len(sig_genes)} genes")

# Prepare data for heatmap
heatmap_data = expr.loc[sig_genes, sample_names].copy()

# Log transform and z-score normalize
heatmap_data_log = np.log2(heatmap_data + 1)
heatmap_data_zscore = heatmap_data_log.T.apply(lambda x: (x - x.mean()) / x.std(), axis=0).T

# Create annotation for samples
sample_colors = {'Control': 'lightblue', 'Stress': 'lightcoral'}
col_colors = [sample_colors[meta_aligned.loc[sample, 'group']] for sample in sample_names]

# Create heatmap
plt.figure(figsize=(12, max(8, len(sig_genes) * 0.3)))

# Create a custom colormap
from matplotlib.colors import LinearSegmentedColormap
colors = ['blue', 'white', 'red']
n_bins = 256
cmap = LinearSegmentedColormap.from_list('custom', colors, N=n_bins)

# Plot heatmap
sns.heatmap(heatmap_data_zscore, 
            cmap=cmap, 
            center=0,
            xticklabels=False,
            yticklabels=True,
            cbar_kws={'label': 'Z-score (log2 expression)'})

# Add sample group annotation at the top
for i, color in enumerate(col_colors):
    plt.axvline(i, ymax=1.02, ymin=1.00, color=color, linewidth=3, clip_on=False)

plt.title('Heatmap of Differentially Expressed Genes\n(Blue bar: Control, Red bar: Stress)')
plt.xlabel('Samples')
plt.ylabel('Genes')
plt.tight_layout()

heatmap_path = os.path.join(RESULTS_DIR, "heatmap_significant_genes.png")
plt.savefig(heatmap_path, dpi=300, bbox_inches='tight')
plt.show()
print(f"Saved: {heatmap_path}")

# --- Summary statistics ---
print(f"\n=== SUMMARY ===")
print(f"Total genes analyzed: {len(res)}")
print(f"Genes with padj < {PADJ_CUTOFF}: {sum(res['padj'] < PADJ_CUTOFF)}")
print(f"Up-regulated genes (padj < {PADJ_CUTOFF}, log2FC >= {FC_CUTOFF}): {sum(up_reg)}")
print(f"Down-regulated genes (padj < {PADJ_CUTOFF}, log2FC <= -{FC_CUTOFF}): {sum(down_reg)}")
print(f"Results saved to: {RESULTS_DIR}/")

Expression matrix shape: (37779, 307)
Metadata shape: (307, 23)
Aligned expression matrix shape: (37779, 307)
Groups: group
Control    154
Stress     153
Name: count, dtype: int64
Control samples: 154
Stress samples: 153
